### Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
<ol>
    <li>Наименование вакансии.
    <li>Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    <li>Ссылку на саму вакансию.
    <li>Сайт, откуда собрана вакансия.
</ol>
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [66]:
def _parser_hh(vacancy):

    vacancy_data = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'bloko-button'})[-2].getText())
    
    last_page = 2
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}).find_all('div', {'class': 'serp-item'})
            
            for item in vacancy_items:
                vacancy_data.append(_parser_item_hh(item))
                
    return vacancy_data

In [128]:
def _parser_item_hh(item):

    vacancy_data = {}
    
    # vacancy_name
    vacancy_name = item.find('a', {'class': 'serp-item__title'}).getText().replace(u'\xa0', u' ')
    
    vacancy_data['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).getText()
    
    vacancy_data['company_name'] = company_name
    
    # city
    city = item.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText().split(', ')[0]
    
    vacancy_data['city'] = city
    
    #salary
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText().replace(u'\u202f', u'')
        
        salary = re.split(r"\s", salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
            salary_currency = salary[2]
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
            salary_currency = salary[2]
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[2])
            salary_currency = salary[3]
        
    vacancy_data['salary_min'] = salary_min
    vacancy_data['salary_max'] = salary_max
    vacancy_data['salary_currency'] = salary_currency
    
    vacancy_link = item.find('a', {'class': 'serp-item__title'})['href']
    
    vacancy_data['vacancy_link'] = vacancy_link 
    
    # site
    vacancy_data['site'] = 'hh.ru'
    
    return vacancy_data

In [129]:
vacancy = 'Python'

vacancy_data = []
vacancy_data.extend(_parser_hh(vacancy))
    
df = pd.DataFrame(vacancy_data)

df

,vacancy_name,company_name,city,salary_min,salary_max,salary_currency,vacancy_link,site
0,Backend программист (Python),MANGO FZCO,Москва,120000.0,NaN,руб.,https://hh.ru/vacancy/77147428?from=vacancy_se...,hh.ru
1,Ведущий инженер по тестированию (python),ЦРТ | Группа компаний,Санкт-Петербург,NaN,NaN,None,https://hh.ru/vacancy/73199414?from=vacancy_se...,hh.ru
2,"Python Developer (проект ""Маркетинг "") Middle",СБЕРКОРУС,Москва,NaN,NaN,None,https://hh.ru/vacancy/76968960?from=vacancy_se...,hh.ru
3,"Python Developer (проект ""Маркетинг БКП"") Middle",СБЕРКОРУС,Москва,NaN,NaN,None,https://hh.ru/vacancy/76978400?from=vacancy_se...,hh.ru
4,"Python Developer (проект ""Маркетинг"") Senior",СБЕРКОРУС,Москва,NaN,NaN,None,https://hh.ru/vacancy/76976985?from=vacancy_se...,hh.ru
5,Разработчик Python,ООО Нетопия,Санкт-Петербург,200000.0,250000.0,руб.,https://hh.ru/vacancy/77091466?from=vacancy_se...,hh.ru
6,Разработчик Python,ООО Нетопия,Москва,200000.0,250000.0,руб.,https://hh.ru/vacancy/77091465?from=vacancy_se...,hh.ru
7,Программист Python Junior,BRANDPOL,Екатеринбург,NaN,NaN,None,https://hh.ru/vacancy/77146976?from=vacancy_se...,hh.ru
8,Программист Python,ООО Траектория,Санкт-Петербург,45000.0,NaN,руб.,https://hh.ru/vacancy/76608111?from=vacancy_se...,hh.ru
9,"Python-разработчик (Django, DRF)",ООО Фабрика Решений,Москва,60000.0,220000.0,руб.,https://hh.ru/vacancy/70705466?from=vacancy_se...,hh.ru
